In [1]:
from models import build_model_bert4Rec
from dataloader import Bert4RecDataLoader

import polars as pl
from tqdm import tqdm
tqdm.pandas()
import gc
from collections import defaultdict
import numpy as np
import pandas as pd
import tensorflow as tf
import os

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

2022-11-28 21:28:09.681107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 21:28:09.741832: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-28 21:28:09.757439: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-28 21:28:10.119091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-11-28 21:28:10.537923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-28 21:28:10.552159: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-28 21:28:10.552261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
df_mapping = pd.read_csv('../tfrecords/tfrecords_v0.5/df_mapping.csv')
NUM_ITEMS = len(df_mapping['aid_map'].unique())

dict_map = {}
for x in tqdm(df_mapping.to_dict('records')):
    dict_map[x['aid_map']] = x['aid']

dict_map_type = {
    'clicks' : 1,
    'carts' : 2,
    'orders' : 3
  }

print(f'NUM_ITEMS: {NUM_ITEMS}')

100%|██████████| 1855603/1855603 [00:00<00:00, 7740021.45it/s]

NUM_ITEMS: 1311743


In [3]:
list_paths = ['../tfrecords/tfrecords_v0.5/na_split=val/' + x for x in os.listdir('../tfrecords/tfrecords_v0.5/na_split=val')]
# 5,45, 1,09
dataloader = Bert4RecDataLoader(list_paths, 
                                num_items=NUM_ITEMS, 
                                seq_len=20, 
                                seq_len_target=None,
                                batch_size=32, 
                                mask_prob=0.4, 
                                reverse_prob=0.25, 
                                get_session=True,
                                is_val=True,
                                is_test=False,
                                shuffle=False).get_generator()
# Val
list_sessions_val = []
for batch in tqdm(dataloader):
    features, targets, session = batch
    list_sessions_val = list_sessions_val + session.numpy().tolist()
print(len(list_sessions_val))
# 20110it [00:27, 720.48it/s]

del dataloader, batch, features, targets, session
gc.collect()

2022-11-28 21:28:20.402515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 21:28:20.403321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-28 21:28:20.403402: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-28 21:28:20.403442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

643503


5727

In [4]:
df_train_all = pl.read_parquet(
    '../0_Data/data_optimized/train.parquet'
)
df_test = pl.read_parquet(
    '../0_Data/data_optimized/test.parquet'
)

# df_train = df_train_all.filter(~(pl.col('session').is_in(list_sessions_val)))
df_val = df_train_all.filter(pl.col('session').is_in(list_sessions_val))

# Validation

## 1. Rule based predictions 

In [10]:
session_types = ['clicks', 'carts', 'orders']

val_session_items = df_val.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
val_session_types = df_val.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
val_sessions = val_session_items.index

type_weight_multipliers = {0: 1, 1: 6, 2: 3}
list_labels, list_sessions = [], []
for session, seq_items, seq_types in tqdm(zip(val_sessions, val_session_items, val_session_types), total=len(val_session_items)):
    if len(seq_items) >= 20:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1, 1, len(seq_items), base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(seq_items, weights, seq_types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        list_labels.append(sorted_aids[:20])
    else:
        list_labels.append([0])
    list_sessions.append(session)

del val_session_items, val_session_types
gc.collect()

df_val_rule = pd.DataFrame({
    'session' : list_sessions,
    'prediction' : list_labels,
})

100%|██████████| 643503/643503 [00:04<00:00, 153577.76it/s]


## 2. Model based predictions

In [12]:
class BERT4REC_CONFIG:
    seed = 42 
    num_items = NUM_ITEMS
    path_tfrecords = '../tfrecords/tfrecords_v0.5/'
    restore_last_chekpoint = (True, 'model_bert4rec_complete_0.10/checkpoints/', 'ckpt-22')
    model_name = 'model_bert4rec_complete_0.10'
    checkpoint_filepath = f'../2_Models/'
    num_records_dataset = 10_000_000
    batch_size = 32
    tup_scheduler_grad_accum = (5, 10, 1_500_000) #(start_grad_accum, max_grad_accum, ramp_up_samples)
    seq_len = 30
    mask_prob = 0.4
    reverse_prob = 0.5
    emb_dim = 128
    trf_dim = 128
    num_heads = 4
    num_layers = 1
    ff_dim = trf_dim*4
    drop_rate = 0.1
    att_drop_rate = 0.1
    epochs = 3
    early_stopping = 5
    batch_num_printer_train = 500
    batch_num_printer_val = 250
    clipnorm = 1.0
    num_iters_save_checkpoint = 25_000
    scheduler_scaler = 128 
    warmup_steps = 10_000
    weight_decay = 1e-1
    log_wandb = True

def get_score_metric(y_true, y_pred, type_target, k=20):
    score = 0 
    if len(y_true)==0:
        return None
    if type_target=='clicks':
        num_targets = 1
        hits = len([x for x in y_pred if x==y_true[0]])
    else:
        num_targets = min(k, len(y_true))
        hits = len([x for x in y_pred if x in y_true])
    score = hits / num_targets
    return score

In [49]:
tf.keras.backend.clear_session()
model = build_model_bert4Rec(num_items=NUM_ITEMS, model_cfg=BERT4REC_CONFIG)
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore(tf.train.latest_checkpoint(f'../2_Models/model_bert4rec_complete_0.10/checkpoints'))
list_paths_val = ['../tfrecords/tfrecords_v0.5/na_split=val/' + x for x in os.listdir('../tfrecords/tfrecords_v0.5/na_split=val')]
val_dataloader = Bert4RecDataLoader(list_paths_val, 
                                     num_items=NUM_ITEMS, 
                                     seq_len=30, 
                                     seq_len_target=30, 
                                     batch_size=32, 
                                     mask_prob=0.0, 
                                     reverse_prob=0.0, 
                                     is_val=True,
                                     get_session=True, 
                                     is_test=False,
                                     shuffle=False).get_generator()


list_sessions, list_past_items, list_predictions, list_trues, list_types = [], [], [], [], []
for num_batch, batch in enumerate(tqdm(val_dataloader)):
    features, targets, session = batch
    seq_items, seq_type, seq_time, seq_recency = features
    target, type_target, idx_mask = targets
    idxs = idx_mask.numpy()
    for type_ in ['clicks', 'carts', 'orders']:
        seq_type_new = [tf.concat([
                        seq_type[i, :ix],
                        tf.constant([[dict_map_type[type_]]], tf.int64),
                        seq_type[i, ix+1:]], axis=0)
                    for i, ix in enumerate(idxs)]
        features = (seq_items, tf.stack(seq_type_new, axis=0), seq_time, seq_recency)
        preds = model(features, training=False)
        preds = tf.gather(preds, indices=idxs, axis=1, batch_dims=1)
        topk_scores, topk_idxs = tf.math.top_k(preds, k=20)
        topk_idxs = np.asarray([[dict_map[x]-1 for x in topk_idxs.numpy()[i, :]] for i in range(topk_idxs.numpy().shape[0])])
        labels = [list(set([dict_map[_target]-1 for _type, _target in zip(type_target.numpy()[i], target.numpy()[i]) if dict_map_type[type_]==_type and _target!=0])) for i in range(target.shape[0])]
        ###
        list_sessions.append(session.numpy())
        list_predictions.append(topk_idxs)
        list_types.append([type_ for _ in range(seq_items.shape[0])])
        list_trues = list_trues + labels
        list_past_items.append(seq_items.numpy()[:, :, 0])
    if num_batch==5_000:
        break

df_val_model = pd.DataFrame({
    'session' : np.concatenate(list_sessions),
    'past_items' : np.concatenate(list_past_items).tolist(),
    'predictions' : np.concatenate(list_predictions).tolist(),
    'trues' : list_trues,
    'type' : np.concatenate(list_types)
})

df_val_model['qt_trues'] = df_val_model['trues'].apply(lambda x : len(x))

5000it [11:09,  7.47it/s]


In [54]:
df_val_merge

,session,prediction,past_items,predictions,trues,type,qt_trues,predictions_final,score
0,94,"[1188425, 818697, 462252, 1607328, 1040888, 18...","[669987, 595366, 1059168, 300686, 875947, 2619...","[959208, 194870, 664490, 1138707, 1506936, 709...","[1317635, 937348, 189208, 1223839, 1729837, 84...",clicks,20,"[1188425, 818697, 462252, 1607328, 1040888, 18...",0.0
1,94,"[1188425, 818697, 462252, 1607328, 1040888, 18...","[669987, 595366, 1059168, 300686, 875947, 2619...","[709987, 194870, 1055626, 1501826, 1546211, 39...","[185144, 461938]",carts,2,"[1188425, 818697, 462252, 1607328, 1040888, 18...",1.0
2,94,"[1188425, 818697, 462252, 1607328, 1040888, 18...","[669987, 595366, 1059168, 300686, 875947, 2619...","[1501826, 470713, 1569605, 709987, 1101316, 19...",[],orders,0,"[1188425, 818697, 462252, 1607328, 1040888, 18...",NaN
3,156,"[123997, 368888, 835255, 1395588, 132129, 1398...","[1192186, 0, 292147, 353566, 973977, 1093217, ...","[1456869, 1483563, 1626136, 1003990, 677909, 8...","[1319681, 1395588, 1546630, 1136391, 1185930, ...",clicks,19,"[123997, 368888, 835255, 1395588, 132129, 1398...",0.0
4,156,"[123997, 368888, 835255, 1395588, 132129, 1398...","[1192186, 0, 292147, 353566, 973977, 1093217, ...","[1483563, 1456869, 627404, 834930, 89925, 1626...",[1395588],carts,1,"[123997, 368888, 835255, 1395588, 132129, 1398...",1.0
...,...,...,...,...,...,...,...,...,...
480091,12899671,[0],"[158569, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[224347, 666582, 293066, 602580, 311287, 18463...",[],carts,0,"[224347, 666582, 293066, 602580, 311287, 18463...",NaN
480092,12899671,[0],"[158569, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[224347, 666582, 293066, 602580, 166037, 29023...",[],orders,0,"[224347, 666582, 293066, 602580, 166037, 29023...",NaN
480093,12899754,[0],"[1016692, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1438223, 231487, 1675253, 1448403, 1820122, 1...",[922732],clicks,1,"[1438223, 231487, 1675253, 1448403, 1820122, 1...",0.0
480094,12899754,[0],"[1016692, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1438223, 1084167, 1387295, 1258519, 1751083, ...",[],carts,0,"[1438223, 1084167, 1387295, 1258519, 1751083, ...",NaN


In [52]:
df_val_merge = df_val_rule.merge(df_val_model, how='inner', on='session')
df_val_merge['predictions_final'] = df_val_merge.progress_apply(lambda x : x['predictions'] if len(x['prediction'])==1 else x['prediction'], axis=1)
df_val_merge['score'] = df_val_merge.progress_apply(lambda x: get_score_metric(x['trues'], x['predictions_final'], x['type']), axis=1)

display(df_val_merge.describe())
dict_scores = df_val_merge.groupby('type')['score'].mean().to_dict()
display(dict_scores)
kaggle_metric = 0.1*dict_scores['clicks'] + 0.3*dict_scores['carts'] + 0.6*dict_scores['orders']
print(f'Kaggle Metric: {kaggle_metric:.4f}')

# # Model
# {'carts': 0.37454512439145493,
#  'clicks': 0.3244825988467875,
#  'orders': 0.5052842557053815}
# Kaggle Metric: 0.4480

# rule
# {'carts': 0.5068184776077022,
#  'clicks': 0.16846427100494235,
#  'orders': 0.5224797601872758}
# Kaggle Metric: 0.4824

# final
# {'carts': 0.7613601893625642,
#  'clicks': 0.4558214065074135,
#  'orders': 0.8298199206770427}
# Kaggle Metric: 0.7719

100%|██████████| 480096/480096 [00:02<00:00, 196862.76it/s]


,session,qt_trues,score
count,4.800960e+05,480096.000000,201701.000000
mean,6.458205e+06,1.544068,0.530755
std,3.719837e+06,3.529577,0.495674
min,9.400000e+01,0.000000,0.000000
25%,3.243194e+06,0.000000,0.000000
50%,6.449878e+06,0.000000,1.000000
75%,9.685902e+06,1.000000,1.000000
max,1.289975e+07,30.000000,1.000000


{'carts': 0.7613601893625642,
 'clicks': 0.4558214065074135,
 'orders': 0.8298199206770427}

Kaggle Metric: 0.7719


# Test